In [1]:
import pygame
from pygame.locals import *
import time
import random


pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
SIZE = 25
BACKGROUND_COLOR = (2,46,117)

class Apple:
    def __init__(self, parent_screen):
        self.parent_screen = parent_screen
        self.image = pygame.image.load("C:\\Users\\Rezwan\\Desktop\\proj\\pic\\apple.PNG").convert()
        self.x = SIZE*4
        self.y = SIZE*4

    def draw(self):
        self.parent_screen.blit(self.image, (self.x, self.y))
        pygame.display.flip()

    def move(self):
        self.x = random.randint(1,19)*SIZE
        self.y = random.randint(1,19)*SIZE

class Snake:
    
    def __init__(self, parent_screen):
        self.parent_screen = parent_screen
        self.image = pygame.image.load("C:\\Users\\Rezwan\\Desktop\\proj\\pic\\r.JPG").convert()
        self.direction = 'down'

        self.length = 1
        self.x = [25]
        self.y = [25]

    def move_left(self):
        self.direction = 'left'

    def move_right(self):
        self.direction = 'right'

    def move_up(self):
        self.direction = 'up'

    def move_down(self):
        self.direction = 'down'

    def walk(self):
        # update body
        for i in range(self.length-1,0,-1):
            self.x[i] = self.x[i-1]
            self.y[i] = self.y[i-1]

        # update head
        if self.direction == 'left':
            self.x[0] -= SIZE
        if self.direction == 'right':
            self.x[0] += SIZE
        if self.direction == 'up':
            self.y[0] -= SIZE
        if self.direction == 'down':
            self.y[0] += SIZE

        self.draw()

    def draw(self):
        for i in range(self.length):
            self.parent_screen.blit(self.image, (self.x[i], self.y[i]))

        pygame.display.flip()

    def increase_length(self):
        self.length += 1
        self.x.append(-1)
        self.y.append(-1)

class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Snake And Apple Game")

        pygame.mixer.init()
        self.play_background_music()

        self.surface = pygame.display.set_mode((800, 500))
        self.snake = Snake(self.surface)
        self.snake.draw()
        self.apple = Apple(self.surface)
        self.apple.draw()

    def play_background_music(self):
        pygame.mixer.music.load('C:\\Users\\Rezwan\\Desktop\\proj\\sound\\piano.mp3')
        pygame.mixer.music.play(-1, 0)

    def play_sound(self, sound_name):
        if sound_name == "Ding":
            sound = pygame.mixer.Sound("C:\\Users\\Rezwan\\Desktop\\proj\\sound\\Ding.mp3")
        elif sound_name == 'game-over':
            sound = pygame.mixer.Sound("C:\\Users\\Rezwan\\Desktop\\proj\\sound\\game-over.mp3")

        pygame.mixer.Sound.play(sound)

    def reset(self):
        self.snake = Snake(self.surface)
        self.apple = Apple(self.surface)

    def is_collision(self, x1, y1, x2, y2):
        if x1 >= x2 and x1 < x2 + SIZE:
            if y1 >= y2 and y1 < y2 + SIZE:
                return True
        return False

    def render_background(self):
        bg = pygame.image.load("C:\\Users\\Rezwan\\Desktop\\proj\\pic\\green.jpg")
        self.surface.blit(bg, (0,0))

    def play(self):
        self.render_background()
        self.snake.walk()
        self.apple.draw()
        self.display_score()
        pygame.display.flip()

        # snake eating apple scenario
        if self.is_collision(self.snake.x[0], self.snake.y[0], self.apple.x, self.apple.y):
            self.play_sound("Ding")
            self.snake.increase_length()
            self.apple.move()

        # snake colliding with itself
        for i in range(2, self.snake.length):
            if self.is_collision(self.snake.x[0], self.snake.y[0], self.snake.x[i], self.snake.y[i]):
                self.play_sound('game-over')
                raise "Collision Occurred"
                
        #snake hit boundary
        if not (0 <= self.snake.x[0] <= 800 and 0 <= self.snake.y[0] <= 500):
            self.play_sound('game-over')
            raise "Hit the boundry error"

    def display_score(self):
        font = pygame.font.SysFont('arial',20)
        score = font.render(f"Score: {self.snake.length}",True,(200,200,200))
        self.surface.blit(score,(650,10))
        
        
    def speed(self):
        if self.snake.length <=10:
                time.sleep(.25)
        elif self.snake.length > 10 and self.snake.length <=20:
                time.sleep(.20)
        elif self.snake.length > 20 and self.snake.length <=30:
                time.sleep(.15)
        elif self.snake.length > 30 and self.snake.length <=40:
                time.sleep(.10)
                
        elif self.snake.length >40 and self.snake.length <= 50:
                time.sleep(.05)
                
        else:
            time.sleep(.005)
        
    def show_game_over(self):
        self.render_background()
        font = pygame.font.SysFont('arial', 20)
        line1 = font.render(f"Game is over! Your score is {self.snake.length}", True, (255, 255, 255))
        self.surface.blit(line1, (200, 200))
        line2 = font.render("To play again press Enter. To exit press Escape!", True, (255, 255, 255))
        self.surface.blit(line2, (200, 250))
        pygame.mixer.music.pause()
        pygame.display.flip()

    def run(self):
        running = True
        pause = False

        while running:
            for event in pygame.event.get():
                if event.type == KEYDOWN:
                    if event.key == K_ESCAPE:
                        running = False

                    if event.key == K_RETURN:
                        pygame.mixer.music.unpause()
                        pause = False

                    if not pause:
                        if event.key == K_LEFT:
                            self.snake.move_left()

                        if event.key == K_RIGHT:
                            self.snake.move_right()

                        if event.key == K_UP:
                            self.snake.move_up()

                        if event.key == K_DOWN:
                            self.snake.move_down()

                elif event.type == QUIT:
                    running = False
            try:

                if not pause:
                    self.play()

            except Exception as e:
                self.show_game_over()
                pause = True
                self.reset()
            
            self.speed()
            
            

if __name__ == '__main__':
    game = Game()
    game.run()